# Data Wrangling Populations
***
Data-source: https://www.worldometers.info/world-population/population-by-country/

Objective: get a clean dataset in SQL ready to merge with the country-based covid data (mind keys for joining). Main requirement is the `population` column, but other parameters could be interesting as well.

In [291]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from src.data.query_db import queryDB
qdb = queryDB('sqlite','../data/processed/covid.sqlite')

%load_ext sql

%load_ext autoreload
%autoreload 2

sqlite:///../data/processed/covid.sqlite
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [267]:
# quick check
qdb.execute_query('SELECT COUNT(*) FROM stats;').iloc[0,0]

1 rows affected


20424

### 1. Gather Data
***
Note: header was already manually adjusted when capturing the data.

In [268]:
df = pd.read_csv('../data/raw/global_pop.csv')
df.head()

,rank,country,populations,yearly_change,net_change,density,land_area,migrants,fert_rate,med_age,urban_pop_pct,world_share_pct
0,1,China,"1,439,323,776",0.39%,"5,540,090",153,"9,388,211","-348,399",1.7,38,61%,18.47%
1,2,India,"1,380,004,385",0.99%,"13,586,631",464,"2,973,190","-532,687",2.2,28,35%,17.70%
2,3,United States,"331,002,651",0.59%,"1,937,734",36,"9,147,420","954,806",1.8,38,83%,4.25%
3,4,Indonesia,"273,523,615",1.07%,"2,898,047",151,"1,811,570","-98,955",2.3,30,56%,3.51%
4,5,Pakistan,"220,892,340",2.00%,"4,327,022",287,"770,880","-233,379",3.6,23,35%,2.83%


In [269]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   rank             235 non-null    int64 
 1   country          235 non-null    object
 2   populations      235 non-null    object
 3   yearly_change    235 non-null    object
 4   net_change       235 non-null    object
 5   density          235 non-null    object
 6   land_area        235 non-null    object
 7   migrants         201 non-null    object
 8   fert_rate        235 non-null    object
 9   med_age          235 non-null    object
 10  urban_pop_pct    235 non-null    object
 11  world_share_pct  235 non-null    object
dtypes: int64(1), object(11)
memory usage: 22.2+ KB


> Note: most of the columns are formatted strings (percentages, commas at 1000). We wish to convert them to numeric columns.

#### Store data in SQL
***
For practise sake, we will do the cleaning in SQL. Hence, we will import the table in it's current state in the SQLite DB and perform the cleaning there.

In [270]:
# store in SQL
table = 'population_raw'
qdb.execute_query('DROP TABLE IF EXISTS {};'.format(table))
df.to_sql(table, con = qdb.engine, if_exists = 'append', index=False, chunksize = 1000)

# check
assert table in qdb.engine.table_names()
assert len(df) == qdb.execute_query('SELECT COUNT(*) FROM {};'.format(table)).iloc[0,0]

unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
1 rows affected


### 2. Data Assessment
***

In [271]:
%%sql sqlite:///../data/processed/covid.sqlite
-- head of the table
SELECT * FROM population_raw LIMIT 5;


Done.


rank,country,populations,yearly_change,net_change,density,land_area,migrants,fert_rate,med_age,urban_pop_pct,world_share_pct
1,China,"1,439,323,776",0.39%,"5,540,090",153,"9,388,211","-348,399",1.7,38,61%,18.47%
2,India,"1,380,004,385",0.99%,"13,586,631",464,"2,973,190","-532,687",2.2,28,35%,17.70%
3,United States,"331,002,651",0.59%,"1,937,734",36,"9,147,420","954,806",1.8,38,83%,4.25%
4,Indonesia,"273,523,615",1.07%,"2,898,047",151,"1,811,570","-98,955",2.3,30,56%,3.51%
5,Pakistan,"220,892,340",2.00%,"4,327,022",287,"770,880","-233,379",3.6,23,35%,2.83%


In [272]:
# SQL data types
qdb.execute_query('PRAGMA table_info(population_raw);')

12 rows affected


,cid,name,type,notnull,dflt_value,pk
0,0,rank,BIGINT,0,None,0
1,1,country,TEXT,0,None,0
2,2,populations,TEXT,0,None,0
3,3,yearly_change,TEXT,0,None,0
4,4,net_change,TEXT,0,None,0
5,5,density,TEXT,0,None,0
6,6,land_area,TEXT,0,None,0
7,7,migrants,TEXT,0,None,0
8,8,fert_rate,TEXT,0,None,0
9,9,med_age,TEXT,0,None,0


In [273]:
%%sql 
-- check the joining keys
SELECT COUNT(DISTINCT stats.country) AS keys_found
  FROM stats
       LEFT JOIN population_raw
       ON population.country = stats.country
 WHERE population.rank IS NOT NULL; 



 * sqlite:///../data/processed/covid.sqlite
(sqlite3.OperationalError) no such column: population.rank
[SQL: -- check the joining keys
SELECT COUNT(DISTINCT stats.country) AS keys_found
  FROM stats
       LEFT JOIN population_raw
       ON population.country = stats.country
 WHERE population.rank IS NOT NULL;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [274]:
%%sql 
-- check the joining keys
SELECT DISTINCT stats.country AS keys_not_found_in_pop 
  FROM stats
       LEFT JOIN population_raw
       ON population_raw.country = stats.country
 WHERE population_raw.rank IS NULL
 ORDER BY 1;  



 * sqlite:///../data/processed/covid.sqlite
Done.


keys_not_found_in_pop
Czech Republic
Ivory Coast
Saint Kitts and Nevis
Saint Vincent and the Grenadines
Sao Tome and Principe


In [275]:
%%sql
-- check the joining keys
SELECT DISTINCT 
       population_raw.country AS keys_not_found_in_stats,
       populations
  FROM population_raw
       LEFT JOIN stats
       ON population_raw.country = stats.country
 WHERE stats.confirmed IS NULL 
 ORDER BY 1;

 * sqlite:///../data/processed/covid.sqlite
Done.


keys_not_found_in_stats,populations
American Samoa,"55,191"
Anguilla,"15,003"
Aruba,"106,766"
Bermuda,"62,278"
British Virgin Islands,"30,231"
Caribbean Netherlands,"26,223"
Cayman Islands,"65,722"
Channel Islands,"173,863"
Cook Islands,"17,564"
Curaçao,"164,093"


#### DATA ASSESSMENT NOTES
**Required changes per column:**
* populations: remove ‘,’ and convert to integer
* yearly_change: remove ‘%’, convert to numeric, change name to ‘yearly_change_pct’
* net_change: remove ‘,’ and convert to integer
* Density: convert to integer
* land_area:  remove ‘,’ and convert to integer
* migrants:  remove ‘,’ and convert to integer
* fert_rate: convert to numeric
* med_age convert to integer
* urban_pop_pct: remove ‘%’ and convert to integer
* world_share_pct: remove ‘%’ and convert to numeric

**With respect to the joining keys:**
* 173 out of 187 are correct!
* For the remaining 14, we can trace back the correct names (i.e. South Korea -> Korea, South)
* We should remove ‘MS Zaandam’ from stats (this is a cruise-ship, not a country)
* No data in stats for the following countries:
    * Most notable exceptions are North Korea and Turkmenistan (most likely due to the lack of transpartent data)
    * Other countries not covered in stats are often subsidiaries of kingdoms (i.e. Caribbean Netherlands, British Virgin Islands, New Caledonia) - we will not pursue further data gathering for these cases.


### 3. Cleaning Data
***

#### CHANGING FORMATS

In [276]:
%%sql
-- create clean populations table
DROP TABLE IF EXISTS populations;

CREATE TABLE populations AS
SELECT
    rank,
    country,
    CAST(REPLACE(populations,',','') AS integer) AS population,
    CAST(REPLACE(yearly_change,'%','') AS numeric) AS yearly_change_pct,
    CAST(REPLACE(net_change,',','') AS integer) AS net_change,
    CAST(density AS integer) AS density,
    CAST(REPLACE(land_area,',','') AS integer) AS land_are,
    CAST(REPLACE(migrants,',','') AS integer) AS migrants,
    CAST(fert_rate AS numeric) AS fert_rate,
    CAST(med_age AS integer) AS med_age,
    CAST(REPLACE(urban_pop_pct,'%','') AS integer) AS urban_pop_pct,
    CAST(REPLACE(world_share_pct,'%','') AS numeric) AS world_share_pct
FROM population_raw;

 * sqlite:///../data/processed/covid.sqlite
Done.
Done.


[]

In [277]:
%%sql
SELECT * FROM populations LIMIT 5;

 * sqlite:///../data/processed/covid.sqlite
Done.


rank,country,population,yearly_change_pct,net_change,density,land_are,migrants,fert_rate,med_age,urban_pop_pct,world_share_pct
1,China,1439323776,0.39,5540090,153,9388211,-348399,1.7,38,61,18.47
2,India,1380004385,0.99,13586631,464,2973190,-532687,2.2,28,35,17.7
3,United States,331002651,0.59,1937734,36,9147420,954806,1.8,38,83,4.25
4,Indonesia,273523615,1.07,2898047,151,1811570,-98955,2.3,30,56,3.51
5,Pakistan,220892340,2,4327022,287,770880,-233379,3.6,23,35,2.83


In [278]:
qdb.execute_query('PRAGMA table_info(populations);')

12 rows affected


,cid,name,type,notnull,dflt_value,pk
0,0,rank,INT,0,None,0
1,1,country,TEXT,0,None,0
2,2,population,INT,0,None,0
3,3,yearly_change_pct,NUM,0,None,0
4,4,net_change,INT,0,None,0
5,5,density,INT,0,None,0
6,6,land_are,INT,0,None,0
7,7,migrants,INT,0,None,0
8,8,fert_rate,NUM,0,None,0
9,9,med_age,INT,0,None,0


#### update countries - basics

In [279]:
%%sql
UPDATE populations
SET country = replace(country, char(39), '')
WHERE country LIKE 'C_te%';

UPDATE stats
SET country = replace(country, char(39), '')
WHERE country LIKE 'C_te%';

 * sqlite:///../data/processed/covid.sqlite
1 rows affected.
0 rows affected.


[]

In [280]:
%%sql
--drop cruiseships & kosovo from stats
DELETE FROM stats 
WHERE country IN ('Diamond Princess','MS Zaandam','Kosovo');

 * sqlite:///../data/processed/covid.sqlite
Done.


[]

In [281]:
%%sql
SELECT COUNT(*)
FROM stats 
WHERE country IN ('Diamond Princess','MS Zaandam','Kosovo');

 * sqlite:///../data/processed/covid.sqlite
Done.


COUNT(*)
0


#### update countries - stats table

In [282]:
def updateCountry(table, original, replacement):
    query = """
        UPDATE {}
        SET country = '{}'
        WHERE country = '{}';
        """.format(table, replacement, original)

    qdb.execute_query(query)

In [283]:
# translation table
trans_stats = {
    'Cote dIvoire' : 'Ivory Coast',
    'Burma' : 'Myanmar',
    'Congo (Brazzaville)' : 'Congo',
    'Congo (Kinshasa)' : 'DR Congo',
    'West Bank and Gaza' : 'State of Palestine',
    'Taiwan*' : 'Taiwan',
    'Czechia' : 'Czech Republic',
    'Korea, South' : 'South Korea'}

# run the update
for key in trans_stats.keys():
    updateCountry('stats', key, trans_stats[key])

unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---


#### update countries - populations table

In [284]:
# translation table
pop_stats = {
    'Czech Republic (Czechia)' : 'Czech Republic',
    'Côte dIvoire' : 'Ivory Coast',
    'Saint Kitts & Nevis' : 'Saint Kitts and Nevis',
    'St. Vincent & Grenadines' : 'Saint Vincent and the Grenadines',
    'Sao Tome & Principe' : 'Sao Tome and Principe'}

# run the update
for key in pop_stats.keys():
    updateCountry('populations', key, pop_stats[key])

unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---


#### check
***
We expect 184 distict countries (187 - 3 removed) in first query and 0 in second.

In [285]:
%%sql 
-- check the joining keys
SELECT COUNT(DISTINCT stats.country) AS cnt 
  FROM stats
       LEFT JOIN populations
       ON populations.country = stats.country
 WHERE populations.rank IS NOT NULL
 ORDER BY 1;  


 * sqlite:///../data/processed/covid.sqlite
Done.


cnt
184


In [287]:
%%sql
-- check the joining keys
SELECT COUNT(DISTINCT stats.country) AS cnt 
  FROM stats
       LEFT JOIN populations
    ON populations.country = stats.country
 WHERE populations.rank IS NULL;  

 * sqlite:///../data/processed/covid.sqlite
Done.


cnt
0


### 4. Calculate the scaled table
***
The overall objective of this dataset is to be able to scale the overall covid-19 numbers to `per 1 million`, hence we need a scaled population (population / 1,000,000) to divide the covid-19 numbers by.

In [229]:
%%sql
ALTER TABLE populations
ADD column scaled_pop numeric;

 * sqlite:///../data/processed/covid.sqlite
Done.


[]

In [233]:
%%sql
UPDATE populations 
   SET scaled_pop = ROUND(population / 1000000.0, 2);

 * sqlite:///../data/processed/covid.sqlite
235 rows affected.


[]

In [234]:
%%sql
SELECT * FROM populations LIMIT 2;

 * sqlite:///../data/processed/covid.sqlite
Done.


rank,country,population,yearly_change_pct,net_change,density,land_are,migrants,fert_rate,med_age,urban_pop_pct,world_share_pct,scaled_pop
1,China,1439323776,0.39,5540090,153,9388211,-348399,1.7,38,61,18.47,1439.32
2,India,1380004385,0.99,13586631,464,2973190,-532687,2.2,28,35,17.7,1380


#### DROP raw table

In [240]:
%%sql
DROP TABLE population_raw;

 * sqlite:///../data/processed/covid.sqlite
Done.


[]

In [241]:
# drop columns we don't need...
qdb.engine.table_names()

['country', 'daily_stats', 'populations', 'stats']

### 5. Automating the process
***
1. add the process above to the pipeline

#### functions

In [325]:
# load
def getPopulationData(table = 'population_raw'):
    """
    Load raw population data into a table
    """
    # load
    df = pd.read_csv('../data/raw/global_pop.csv')

    # store in SQL
    qdb.execute_query('DROP TABLE IF EXISTS {};'.format(table))
    df.to_sql(table, con = qdb.engine, if_exists = 'append', index=False, chunksize = 1000)

    # check
    assert table in qdb.engine.table_names()
    assert len(df) == qdb.execute_query('SELECT COUNT(*) FROM {};'.format(table)).iloc[0,0]
    

def updateCountry(table, original, replacement):
    """
    Change country from original to replacement in table
    """
    # clean countries based on hash table
    qdb.execute_query("""
        UPDATE {}
           SET country = '{}'
         WHERE country = '{}';""".format(table, replacement, original))
    

    
# clean (queries) -> SQL clean
def cleanPopulation(raw_table = 'population_raw', target_table = 'populations', 
                    country_trans_file = '../src/data/clean_country_names.json'):
    """
    Create populations table containing the cleaned population data
    """
    # clean data in raw table and store it in target table  
    if raw_table in qdb.engine.table_names():
        qdb.execute_query('DROP TABLE IF EXISTS {};'.format(target_table))
        qdb.execute_query("""
            CREATE TABLE {} AS
            SELECT rank,
                   country,
                   CAST(REPLACE(populations,',','') AS integer) AS population,
                   CAST(REPLACE(yearly_change,'%','') AS numeric) AS yearly_change_pct,
                   CAST(REPLACE(net_change,',','') AS integer) AS net_change,
                   CAST(density AS integer) AS density,
                   CAST(REPLACE(land_area,',','') AS integer) AS land_are,
                   CAST(REPLACE(migrants,',','') AS integer) AS migrants,
                   CAST(fert_rate AS numeric) AS fert_rate,
                   CAST(med_age AS integer) AS med_age,
                   CAST(REPLACE(urban_pop_pct,'%','') AS integer) AS urban_pop_pct,
                   CAST(REPLACE(world_share_pct,'%','') AS numeric) AS world_share_pct
              FROM population_raw;""".format(target_table))
        
        # remove invalid countries or countries without population data from stats table
        qdb.execute_query("""
            DELETE FROM stats 
             WHERE country IN ('Diamond Princess','MS Zaandam','Kosovo');""")
        
        assert qdb.execute_query("""
            SELECT COUNT(*)
              FROM stats 
             WHERE country IN ('Diamond Princess','MS Zaandam','Kosovo');""").iloc[0,0] == 0
        
        # open the country translation table
        with open(country_trans_file) as f:
            country_trans = json.loads(f.read())
            
        # clean countries in stats table 
        for table in ['stats', 'populations']:
            # remove apostrophe in 'Core d'Ivor'
            qdb.execute_query("""
                UPDATE {}
                   SET country = replace(country, char(39), '')
                 WHERE country LIKE 'C_te%';""".format(table))
            
            # update individual countries
            for key in country_trans[table].keys():
                updateCountry(table, key, country_trans[table][key])
        
        # check country conversion (missing keys..)
        assert qdb.execute_query("""
                SELECT COUNT(DISTINCT stats.country) AS cnt 
                  FROM stats
                       LEFT JOIN populations
                    ON populations.country = stats.country
                 WHERE populations.rank IS NULL;""").iloc[0,0] == 0 # update
      
        # drop SQL raw
        qdb.execute_query('DROP TABLE IF EXISTS {};'.format(raw_table))

    else:
        print('error: raw data not available')
    
    

def calculateScaledPopulation(target_table = 'populations', denominator = 1000000):
    """
    Add a scaled population column: population/1,000,000
    """
    # create the column for the scaled population
    qdb.execute_query("""
        ALTER TABLE populations
          ADD COLUMN scaled_pop numeric;""".format(target_table))
    
    # calculate and store the scaled population
    qdb.execute_query("""
        UPDATE {} 
           SET scaled_pop = ROUND(population / {}.0, 2);""".format(target_table, denominator))

#### store country translation table

In [323]:
# store the json
country_trans = {
    'stats' : {
        'Cote dIvoire' : 'Ivory Coast',
        'Burma' : 'Myanmar',
        'Congo (Brazzaville)' : 'Congo',
        'Congo (Kinshasa)' : 'DR Congo',
        'West Bank and Gaza' : 'State of Palestine',
        'Taiwan*' : 'Taiwan',
        'Czechia' : 'Czech Republic',
        'Korea, South' : 'South Korea'},
    'populations' : {
        'Czech Republic (Czechia)' : 'Czech Republic',
        'Côte dIvoire' : 'Ivory Coast',
        'Saint Kitts & Nevis' : 'Saint Kitts and Nevis',
        'St. Vincent & Grenadines' : 'Saint Vincent and the Grenadines',
        'Sao Tome & Principe' : 'Sao Tome and Principe'}}
    
with open('../src/data/clean_country_names.json', 'w') as fp:
    json.dump(country_trans, fp)

In [324]:
# quick test
with open('../src/data/clean_country_names.json') as f:
    data = json.loads(f.read())


#### End-to-end test

In [332]:
%%sql
-- start by dropping the current table
DROP TABLE populations;
DROP TABLE population_raw;

 * sqlite:///../data/processed/covid.sqlite
(sqlite3.OperationalError) no such table: populations
[SQL: -- start by dropping the current table
DROP TABLE populations;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [334]:
qdb.engine.table_names()

['country', 'daily_stats', 'populations', 'stats']

In [329]:
# insert raw population table
getPopulationData(table = 'population_raw')

# clean the population table
cleanPopulation(raw_table = 'population_raw', target_table = 'populations', 
                    country_trans_file = 'clean_country_names.json')

# calculate the scaled population column
calculateScaledPopulation(target_table = 'populations', denominator = 1000000)

unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
1 rows affected
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
1 rows affected
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unable to execute query
---
This result object does not return rows. It has been closed automatically.
---
unabl

In [336]:
%%sql 
-- test the final table
SELECT * 
  FROM populations
 LIMIT 5;

 * sqlite:///../data/processed/covid.sqlite
Done.


rank,country,population,yearly_change_pct,net_change,density,land_are,migrants,fert_rate,med_age,urban_pop_pct,world_share_pct,scaled_pop
1,China,1439323776,0.39,5540090,153,9388211,-348399,1.7,38,61,18.47,1439.32
2,India,1380004385,0.99,13586631,464,2973190,-532687,2.2,28,35,17.7,1380
3,United States,331002651,0.59,1937734,36,9147420,954806,1.8,38,83,4.25,331
4,Indonesia,273523615,1.07,2898047,151,1811570,-98955,2.3,30,56,3.51,273.52
5,Pakistan,220892340,2,4327022,287,770880,-233379,3.6,23,35,2.83,220.89
